<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [2]:
!pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown

In [4]:
!pip install ipykernel

In [1]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\HP\VNP-Exercises\Labs\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<?, ?B/s]


### Import the required libraries

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, GridSearchCV, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [15]:
data = pd.read_csv('ElectricCarData.csv')

In [27]:
data.sample(10)

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
34,Mazda,MX-30,9.0,150,180,178,240,Yes,FWD,Type 2 CCS,SUV,C,5,32646
70,BMW,i3s 120 Ah,6.9,160,230,165,260,Yes,RWD,Type 2 CCS,Hatchback,B,4,41526
33,Tesla,Cybertruck Tri Motor,3.0,210,750,267,710,Yes,AWD,Type 2 CCS,Pickup,N,6,75000
95,Volkswagen,ID.3 Pro Performance,7.3,160,340,171,470,Yes,RWD,Type 2 CCS,Hatchback,C,5,35575
64,Ford,Mustang Mach-E ER RWD,7.0,180,450,200,430,Yes,RWD,Type 2 CCS,SUV,D,5,54475
17,Volkswagen,e-Up!,11.9,130,195,166,170,Yes,FWD,Type 2 CCS,Hatchback,A,4,21421
52,Opel,Mokka-e,8.5,150,255,176,390,Yes,FWD,Type 2 CCS,SUV,B,5,35000
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
15,Volkswagen,ID.3 Pro S,7.9,160,440,175,590,Yes,RWD,Type 2 CCS,Hatchback,C,4,40936
53,Skoda,Enyaq iV 80,8.8,160,420,183,560,Yes,RWD,Type 2 CCS,SUV,C,5,40000


### Encode string variables

In [47]:
def label_data(data:pd.DataFrame, columns:list):
  encoder = LabelEncoder()
  data_copy = data.copy()

  for column in columns:
    data_copy[column] = encoder.fit_transform(data_copy[[column]].astype(str).values.ravel())
    
  return data_copy

In [51]:
data.BodyStyle.value_counts()

BodyStyle
SUV          45
Hatchback    32
Sedan        10
Liftback      5
Pickup        3
Cabrio        3
SPV           3
MPV           1
Station       1
Name: count, dtype: int64

In [53]:
data = label_data(data=data, columns=['Brand', 'Model', 'RapidCharge', 'PlugType', 'BodyStyle'])

In [55]:
power_encode = {
    'AWD': 0,
    'RWD': 1,
    'FWD': 2
}

In [57]:
def encoder_power(power):
    return power_encode.get(power)

In [59]:
data['PowerTrain'] = data['PowerTrain'].apply(encoder_power)

In [63]:
data.Segment.unique()

array(['D', 'C', 'B', 'F', 'A', 'E', 'N', 'S'], dtype=object)

In [65]:
segment_encode = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'N': 6,
    'S': 7
}

In [67]:
def encoder_segment(segment):
    return segment_encode.get(segment)

In [69]:
data['Segment'] = data['Segment'].apply(encoder_segment)

In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Brand            103 non-null    int32  
 1   Model            103 non-null    int32  
 2   AccelSec         103 non-null    float64
 3   TopSpeed_KmH     103 non-null    int64  
 4   Range_Km         103 non-null    int64  
 5   Efficiency_WhKm  103 non-null    int64  
 6   FastCharge_KmH   103 non-null    object 
 7   RapidCharge      103 non-null    int32  
 8   PowerTrain       103 non-null    int64  
 9   PlugType         103 non-null    int32  
 10  BodyStyle        103 non-null    int32  
 11  Segment          103 non-null    int64  
 12  Seats            103 non-null    int64  
 13  PriceEuro        103 non-null    int64  
dtypes: float64(1), int32(5), int64(7), object(1)
memory usage: 9.4+ KB


In [87]:
data['FastCharge_KmH'].unique()

array(['940', '250', '620', '560', '190', '220', '420', '650', '540',
       '440', '230', '380', '210', '590', '780', '170', '260', '930',
       '850', '910', '490', '470', '270', '450', '350', '710', '240',
       '390', '570', '610', '340', '730', '920', '-', '550', '900', '520',
       '430', '890', '410', '770', '460', '360', '810', '480', '290',
       '330', '740', '510', '320', '500'], dtype=object)

In [89]:
data['FastCharge_KmH'] = data['FastCharge_KmH'].replace('-', np.nan)

# Convert to numeric (float or integer), forcing invalid entries to NaN
data['FastCharge_KmH'] = pd.to_numeric(data['FastCharge_KmH'], errors='coerce')

# Optionally, fill NaN with 0 or another default value
data['FastCharge_KmH'] = data['FastCharge_KmH'].fillna(data['FastCharge_KmH'].mean()).astype(int)

In [91]:
data['FastCharge_KmH'].unique()

array([940, 250, 620, 560, 190, 220, 420, 650, 540, 440, 230, 380, 210,
       590, 780, 170, 260, 930, 850, 910, 490, 470, 270, 450, 350, 710,
       240, 390, 570, 610, 340, 730, 920, 456, 550, 900, 520, 430, 890,
       410, 770, 460, 360, 810, 480, 290, 330, 740, 510, 320, 500])

## Split the dataset for training and testing in ratio 80:20

In [93]:
X, Y = data.drop(columns=['PriceEuro'], axis=1), data['PriceEuro']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [228]:
tree_model = DecisionTreeRegressor(max_depth=7, min_samples_split=3, max_features='sqrt')
tree_model.fit(X_train, Y_train)

DecisionTreeRegressor(max_depth=7, max_features='sqrt', min_samples_split=3)

## Predict the outcomes for X test

In [230]:
y_pred_tree = tree_model.predict(X_test)
y_pred_tree

array([ 68040.        ,  80326.66666667,  55000.        ,  39453.66666667,
        57750.        ,  46250.        ,  69498.83333333,  65000.        ,
       137500.        ,  80326.66666667,  46380.        , 137500.        ,
        69498.83333333,  57750.        ,  55000.        ,  37068.85714286,
        65000.        ,  29972.375     , 137500.        ,  37068.85714286,
        33120.33333333])

## Assess the model performance, by using sklearn metrics for regression

In [404]:
mse_tree = mean_squared_error(Y_test, y_pred_tree)
mae_tree = mean_absolute_error(Y_test, y_pred_tree)
r2_tree = r2_score(Y_test, y_pred_tree)

print("R-squared (R²):", r2_tree)
print("Mean Absolute Error (MAE):", mae_tree)
print("Mean Squared Error (MSE):", mse_tree)

R-squared (R²): 0.8324392390703015
Mean Absolute Error (MAE): 11433.972505668937
Mean Squared Error (MSE): 251246457.44401413


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [398]:
xgb_model = XGBRegressor(n_estimators = 30, max_depth=4, learning_rate=0.1, objective='reg:squarederror', random_state = 42)
xgb_model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=30, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

## Predict the outcomes for X test

In [400]:
y_pred_xgb = xgb_model.predict(X_test)
y_pred_xgb

array([ 55110.42 ,  87479.27 ,  55984.383,  39624.008,  48056.934,
        47397.715,  58114.773,  60900.914, 123867.52 , 100000.97 ,
        60740.78 , 126449.695,  55676.875,  43392.56 ,  55708.047,
        39006.71 ,  60344.293,  30310.88 , 127209.21 ,  33913.35 ,
        34002.14 ], dtype=float32)

## Assess the model performance, by using sklearn metrics for regression

In [402]:
mse = mean_squared_error(Y_test, y_pred_xgb)
mae = mean_absolute_error(Y_test, y_pred_xgb)
r2 = r2_score(Y_test, y_pred_xgb)

print("R-squared (R²):", r2)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)

R-squared (R²): 0.8255207538604736
Mean Absolute Error (MAE): 10327.052734375
Mean Squared Error (MSE): 261620306.66074207


## Compare the performances of both model for at least three regression metircs

In [408]:
print("Decision Tree Regressor:\nR-squared (R²):", r2_tree)
print("Mean Absolute Error (MAE):", mae_tree)
print("Mean Squared Error (MSE):", mse_tree)
print("\nXGB Regressor:\nR-squared (R²):", r2)
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)

Decision Tree Regressor:
R-squared (R²): 0.8324392390703015
Mean Absolute Error (MAE): 11433.972505668937
Mean Squared Error (MSE): 251246457.44401413

XGB Regressor:
R-squared (R²): 0.8255207538604736
Mean Absolute Error (MAE): 10327.052734375
Mean Squared Error (MSE): 261620306.66074207
